In [ ]:
import random
import json
import pandas as pd
from datetime import datetime, timedelta, date # Import date explicitly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline # Import pipeline here
import torch

# === Constants ===

CSV_FILE = 'excuse_history.csv'
DISCLAIMER = "Disclaimer: This chat log is fictional and generated by an AI. It should not be used for deceptive purposes."

# === Excuse Data Structure ===
class ExcuseData:
    def __init__(self, context, tone, excuse, apology, proofs=None, calls=0, effectiveness=None):
        self.context = context
        self.tone = tone
        self.excuse = excuse
        self.apology = apology
        self.proofs = proofs if proofs is not None else {}
        self.calls = calls
        self.effectiveness = effectiveness

    def to_dict(self):
        return {
            "Context": self.context,
            "Tone": self.tone,
            "Excuse": self.excuse,
            "Apology": self.apology,
            "Proofs": json.dumps(self.proofs), # Convert proofs dictionary to JSON string
            "Calls": self.calls,
            "Effectiveness": self.effectiveness
        }

# === Load FLAN-T5 Model ===
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# === FLAN-T5 Excuse Generator ===
def generate_excuses(situation, num_excuses=5, max_length=50):
    prompt = f"Give {num_excuses} creative excuses for the following situation: {situation}"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_excuses,
            temperature=0.9,
            do_sample=True
        )

    excuses = []
    for output in outputs:
        decoded = tokenizer.decode(output, skip_special_tokens=True)
        # Prefix disclaimer
        excuses.append(f" {decoded}")
    return excuses

# Excuse History & Ranking (using the ExcuseData structure)
excuse_history_data = [] # Use this list to store ExcuseData objects for the current run

def add_excuse_data(excuse_data):
    excuse_history_data.append(excuse_data)

def rank_excuses_data(excuse_list): # Modified to accept a list
    # Rank based on the effectiveness stored in ExcuseData objects
    return sorted(excuse_list, key=lambda x: x.effectiveness if x.effectiveness is not None else -1, reverse=True)



# === Fictional Chat Generator ===
def generate_fake_chat(excuse):
    # Return the chat log data instead of printing
    return [
        {"sender": "Mom", "time": "8:42 AM", "message": "Where are you?? Your professor called 😡"},
        {"sender": "You", "time": "8:43 AM", "message": excuse},
        {"sender": "Mom", "time": "8:43 AM", "message": "Send a picture right now."},
        {"sender": "You", "time": "8:44 AM", "message": "Sent you one just now, look at the jam. I’ll be there ASAP."},
        {"sender": "System", "time": "Generated Now", "message": DISCLAIMER}
    ]


# === Medical Certificate Generator ===
def generate_fake_medical_note(patient_name="John Doe", reason=""):
    today = date.today() # Use date.today()
    issue_date = today.strftime("%B %d, %Y")
    rest_days = random.randint(2, 7)

    certificate_text = f"""
------------------------------------------------------------
                         MEDICAL CERTIFICATE
------------------------------------------------------------

To Whom It May Concern,

This is to certify that {patient_name} has been under my care and treatment
for the following medical reason: {reason}. The patient reported symptoms
requiring clinical attention and was advised to take adequate rest.

It is hereby recommended that {patient_name} be granted leave for a period
of {rest_days} days, starting from {issue_date}, in order to facilitate recovery.

Should you have any questions, you may contact my clinic directly.

Sincerely,


M.B.B.S., M.D.
Reg. No: {random.randint(100000, 999999)}
Clinic: Health First Multispeciality, Hyderabad
Date Issued: {issue_date}
------------------------------------------------------------
"""
    return certificate_text

# === Fake Location Log ===
def generate_fake_location_log():
    # Return the location log data instead of printing
    now = datetime.now()
    locations = [
        {
            "timestamp": now.isoformat(),
            "latitude": 12.9716,
            "longitude": 77.5946,

        },
        {
            "timestamp": (now + timedelta(minutes=90)).isoformat(),
            "latitude": 12.9353,
            "longitude": 77.6143,

        }
    ]
    return {

        "location_log": locations
    }

# === Guilt-Tripping Apology Generator (using pipeline from previous cells) ===
apology_generator = pipeline("text-generation", model="gpt2")

def generate_apology(context, tone="emotional"):
    prompt = f"I'm deeply sorry for {context}. I feel really {tone} about it because"
    # Ensure prompt is within a reasonable length for the model
    if len(prompt) > tokenizer.model_max_length:
        prompt = prompt[:tokenizer.model_max_length]

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    # Check if pad_token_id is None and set it to eos_token_id if it is
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Create attention mask
    attention_mask = inputs['input_ids'].ne(tokenizer.pad_token_id)

    with torch.no_grad():
        apology_outputs = model.generate(
            **inputs,
            max_length=50, # Limit apology length
            num_return_sequences=1,
            temperature=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            # attention_mask=attention_mask # Removed redundant attention mask
        )
    apology = tokenizer.decode(apology_outputs[0], skip_special_tokens=True)
    # Remove the prompt from the generated text
    apology = apology[len(prompt):].strip()
    return apology


# === CSV Handling Functions ===
def load_excuses_from_csv(filepath=CSV_FILE):
    try:
        df = pd.read_csv(filepath)
        # Convert the 'Proofs' column from JSON string back to dictionary
        if 'Proofs' in df.columns:
            df['Proofs'] = df['Proofs'].apply(lambda x: json.loads(x) if pd.notnull(x) else {})
        return df
    except FileNotFoundError:
        return pd.DataFrame(columns=["Context", "Tone", "Excuse", "Apology", "Proofs", "Calls", "Effectiveness"])

def save_excuses_to_csv(df, filepath=CSV_FILE):
    # Convert the 'Proofs' column (dictionaries) to JSON strings before saving
    if 'Proofs' in df.columns:
        df['Proofs'] = df['Proofs'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)
    df.to_csv(filepath, index=False)
    print(f"\nExcuse data saved to {filepath}")

# === Main Program ===
if __name__ == "__main__":
    print("=== FLAN-T5 Excuse + Fake Proof Generator & CSV Saver ===")

    # Load existing data from CSV
    all_excuses_df = load_excuses_from_csv()
    print(f"Loaded {len(all_excuses_df)} existing excuses from {CSV_FILE}")

    # Get user inputs for scenario and tone
    scenario = input("Enter a situation (e.g., I missed the test): ")
    # Add input for tone, as used in apology generation
    tone = input("What tone should the apology have? (casual, formal, emotional - default emotional): ") or "emotional"


    # Generate excuses with FLAN-T5
    generated_excuses_text = generate_excuses(scenario, num_excuses=5)

    print(f"\n📝 Generated {len(generated_excuses_text)} Excuses:")
    current_run_excuses = [] # List to hold ExcuseData objects for the current run

    for i, excuse_text in enumerate(generated_excuses_text):
        # Generate apology for each excuse (using the scenario as context)
        apology_text = generate_apology(scenario, tone)

        # Create an ExcuseData object for each generated excuse
        excuse_data = ExcuseData(
            context=scenario,
            tone=tone,
            excuse=excuse_text,
            apology=apology_text,
            effectiveness=random.uniform(0.5, 1.0) # Assign random effectiveness for now
        )
        current_run_excuses.append(excuse_data)
        print(f"{i+1}. {excuse_text}")

    # Rank the excuses from the current run
    ranked_current_excuses = rank_excuses_data(current_run_excuses) # Pass the list to rank_excuses_data
    print("\n🏅 Ranked Excuses (Current Run):")
    for i, excuse_data in enumerate(ranked_current_excuses):
         print(f"{i+1}. Effectiveness: {excuse_data.effectiveness:.2f} - {excuse_data.excuse}")

    # Ask user to select an excuse for proofs
    selected_excuse_data = None
    while True:
        try:
            selection = int(input(f"\nSelect an excuse number (1-{len(current_run_excuses)}) from the GENERATED list above to generate proofs for, or enter 0 to skip proofs: "))
            if 0 <= selection <= len(current_run_excuses):
                break
            else:
                print("Invalid selection. Please enter a number within the range.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    new_excuses_to_save_dicts = [] # List to hold dictionaries of selected excuse for saving
    if selection > 0:
        selected_excuse_data = current_run_excuses[selection - 1]
        # print(f"Type of selected_excuse_data: {type(selected_excuse_data)}") # Added for debugging
        print(f"\nGenerating proofs for selected excuse: {selected_excuse_data.excuse}")

        proofs = {}
        # Chat log
        if input("Do you want a fake chat log for this excuse? (yes/no): ").strip().lower() == 'yes':
            chat_log_data = generate_fake_chat(selected_excuse_data.excuse)
            proofs['chat_log'] = chat_log_data # Store the data
            print("\n📱 Chat Log:")
            for msg in chat_log_data:
                print(f"[{msg['sender']} - {msg['time']}] {msg['message']}")

        # Medical note
        if input("Do you want a fake medical note? (yes/no): ").strip().lower() == 'yes':
            medical_note_text = generate_fake_medical_note()
            proofs['medical_note'] = medical_note_text # Store the text
            print("\n📄 Medical Note:")
            print(medical_note_text)

        # Location log
        if input("Do you want a fake location log? (yes/no): ").strip().lower() == 'yes':
            location_log_data = generate_fake_location_log()
            proofs['location_log'] = location_log_data # Store the data
            print("\n📍 Location Log (JSON):")
            print(json.dumps(location_log_data, indent=4))

        # Update the selected ExcuseData object with generated proofs
        selected_excuse_data.proofs = proofs
        new_excuses_to_save_dicts.append(selected_excuse_data.to_dict()) # Add selected excuse to the list for saving

    else:
        print("\nSkipping proof generation and saving for this run.")

    # Convert the selected excuse data to a DataFrame (will be empty if no excuse was selected)
    new_excuses_df = pd.DataFrame(new_excuses_to_save_dicts)

    # Append new data to the existing DataFrame
    all_excuses_df = pd.concat([all_excuses_df, new_excuses_df], ignore_index=True)

    # Save the combined DataFrame back to CSV
    save_excuses_to_csv(all_excuses_df)

    # Optional: Display the full history from the CSV
    print("\n=== Full Excuse History from CSV ===")
    display(load_excuses_from_csv())

Device set to use cpu


=== FLAN-T5 Excuse + Fake Proof Generator & CSV Saver ===
Loaded 0 existing excuses from excuse_history.csv
Enter a situation (e.g., I missed the test): missed project deadline 
What tone should the apology have? (casual, formal, emotional - default emotional): emotional

📝 Generated 5 Excuses:
1.  I'm trying to be smarter.
2.  I was working late because I had a meeting with a client
3.  i'm lazy , i forgot to pay bills and that is the reason i'm late
4.  It was too many things to complete
5.  I had to travel for work and I got stuck at home

🏅 Ranked Excuses (Current Run):
1. Effectiveness: 0.98 -  It was too many things to complete
2. Effectiveness: 0.97 -  i'm lazy , i forgot to pay bills and that is the reason i'm late
3. Effectiveness: 0.96 -  I had to travel for work and I got stuck at home
4. Effectiveness: 0.93 -  I was working late because I had a meeting with a client
5. Effectiveness: 0.62 -  I'm trying to be smarter.

Select an excuse number (1-5) from the GENERATED list ab

/tmp/ipython-input-3436656448.py:285: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_excuses_df = pd.concat([all_excuses_df, new_excuses_df], ignore_index=True)


,Context,Tone,Excuse,Apology,Proofs,Calls,Effectiveness
0,missed project deadline,emotional,I'm trying to be smarter.,NaN,{},0,0.618033


In [ ]:
import random
import json
import pandas as pd
from datetime import datetime, timedelta, date # Import date explicitly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline # Import pipeline here
import torch

# === Constants ===

CSV_FILE = 'excuse_history.csv'
DISCLAIMER = "Disclaimer: This chat log is fictional and generated by an AI. It should not be used for deceptive purposes."

# === Excuse Data Structure ===
class ExcuseData:
    def __init__(self, context, tone, excuse, apology, proofs=None, calls=0, effectiveness=None):
        self.context = context
        self.tone = tone
        self.excuse = excuse
        self.apology = apology
        self.proofs = proofs if proofs is not None else {}
        self.calls = calls
        self.effectiveness = effectiveness

    def to_dict(self):
        return {
            "Context": self.context,
            "Tone": self.tone,
            "Excuse": self.excuse,
            "Apology": self.apology,
            "Proofs": json.dumps(self.proofs), # Convert proofs dictionary to JSON string
            "Calls": self.calls,
            "Effectiveness": self.effectiveness
        }

# === Load FLAN-T5 Model ===
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# === FLAN-T5 Excuse Generator ===
def generate_excuses(situation, num_excuses=5, max_length=50):
    prompt = f"Give {num_excuses} creative excuses for the following situation: {situation}"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_excuses,
            temperature=0.9,
            do_sample=True
        )

    excuses = []
    for output in outputs:
        decoded = tokenizer.decode(output, skip_special_tokens=True)
        # Prefix disclaimer
        excuses.append(f" {decoded}")
    return excuses

# Excuse History & Ranking (using the ExcuseData structure)
excuse_history_data = [] # Use this list to store ExcuseData objects for the current run

def add_excuse_data(excuse_data):
    excuse_history_data.append(excuse_data)

def rank_excuses_data(excuse_list): # Modified to accept a list
    # Rank based on the effectiveness stored in ExcuseData objects
    return sorted(excuse_list, key=lambda x: x.effectiveness if x.effectiveness is not None else -1, reverse=True)



# === Fictional Chat Generator ===
def generate_fake_chat(excuse):
    # Return the chat log data instead of printing
    return [
        {"sender": "Mom", "time": "8:42 AM", "message": "Where are you?? Your professor called 😡"},
        {"sender": "You", "time": "8:43 AM", "message": excuse},
        {"sender": "Mom", "time": "8:43 AM", "message": "Send a picture right now."},
        {"sender": "You", "time": "8:44 AM", "message": "Sent you one just now, look at the jam. I’ll be there ASAP."},
        {"sender": "System", "time": "Generated Now", "message": DISCLAIMER}
    ]


# === Medical Certificate Generator ===
def generate_fake_medical_note(patient_name="John Doe", reason=""):
    today = date.today() # Use date.today()
    issue_date = today.strftime("%B %d, %Y")
    rest_days = random.randint(2, 7)

    certificate_text = f"""
------------------------------------------------------------
                         MEDICAL CERTIFICATE
------------------------------------------------------------

To Whom It May Concern,

This is to certify that {patient_name} has been under my care and treatment
for the following medical reason: {reason}. The patient reported symptoms
requiring clinical attention and was advised to take adequate rest.

It is hereby recommended that {patient_name} be granted leave for a period
of {rest_days} days, starting from {issue_date}, in order to facilitate recovery.

Should you have any questions, you may contact my clinic directly.

Sincerely,


M.B.B.S., M.D.
Reg. No: {random.randint(100000, 999999)}
Clinic: Health First Multispeciality, Hyderabad
Date Issued: {issue_date}
------------------------------------------------------------
"""
    return certificate_text

# === Fake Location Log ===
def generate_fake_location_log():
    # Return the location log data instead of printing
    now = datetime.now()
    locations = [
        {
            "timestamp": now.isoformat(),
            "latitude": 12.9716,
            "longitude": 77.5946,

        },
        {
            "timestamp": (now + timedelta(minutes=90)).isoformat(),
            "latitude": 12.9353,
            "longitude": 77.6143,

        }
    ]
    return {

        "location_log": locations
    }

# === Guilt-Tripping Apology Generator (using pipeline from previous cells) ===
apology_generator = pipeline("text-generation", model="gpt2")

def generate_apology(context, tone="emotional"):
    prompt = f"I'm deeply sorry for {context}. I feel really {tone} about it because"
    # Ensure prompt is within a reasonable length for the model
    if len(prompt) > tokenizer.model_max_length:
        prompt = prompt[:tokenizer.model_max_length]

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    # Check if pad_token_id is None and set it to eos_token_id if it is
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Create attention mask
    attention_mask = inputs['input_ids'].ne(tokenizer.pad_token_id)

    with torch.no_grad():
        apology_outputs = model.generate(
            **inputs,
            max_length=50, # Limit apology length
            num_return_sequences=1,
            temperature=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            # attention_mask=attention_mask # Removed redundant attention mask
        )
    apology = tokenizer.decode(apology_outputs[0], skip_special_tokens=True)
    # Remove the prompt from the generated text
    apology = apology[len(prompt):].strip()
    return apology


# === CSV Handling Functions ===
def load_excuses_from_csv(filepath=CSV_FILE):
    try:
        df = pd.read_csv(filepath)
        # Convert the 'Proofs' column from JSON string back to dictionary
        if 'Proofs' in df.columns:
            df['Proofs'] = df['Proofs'].apply(lambda x: json.loads(x) if pd.notnull(x) else {})
        return df
    except FileNotFoundError:
        return pd.DataFrame(columns=["Context", "Tone", "Excuse", "Apology", "Proofs", "Calls", "Effectiveness"])

def save_excuses_to_csv(df, filepath=CSV_FILE):
    # Convert the 'Proofs' column (dictionaries) to JSON strings before saving
    if 'Proofs' in df.columns:
        df['Proofs'] = df['Proofs'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)
    df.to_csv(filepath, index=False)
    print(f"\nExcuse data saved to {filepath}")

# === Main Program ===
if __name__ == "__main__":
    print("=== FLAN-T5 Excuse + Fake Proof Generator & CSV Saver ===")

    # Load existing data from CSV
    all_excuses_df = load_excuses_from_csv()
    print(f"Loaded {len(all_excuses_df)} existing excuses from {CSV_FILE}")

    # Get user inputs for scenario and tone
    scenario = input("Enter a situation (e.g., I missed the test): ")
    # Add input for tone, as used in apology generation
    tone = input("What tone should the apology have? (casual, formal, emotional - default emotional): ") or "emotional"


    # Generate excuses with FLAN-T5
    generated_excuses_text = generate_excuses(scenario, num_excuses=5)

    print(f"\n📝 Generated {len(generated_excuses_text)} Excuses:")
    current_run_excuses = [] # List to hold ExcuseData objects for the current run

    for i, excuse_text in enumerate(generated_excuses_text):
        # Generate apology for each excuse (using the scenario as context)
        apology_text = generate_apology(scenario, tone)

        # Create an ExcuseData object for each generated excuse
        excuse_data = ExcuseData(
            context=scenario,
            tone=tone,
            excuse=excuse_text,
            apology=apology_text,
            effectiveness=random.uniform(0.5, 1.0) # Assign random effectiveness for now
        )
        current_run_excuses.append(excuse_data)
        print(f"{i+1}. {excuse_text}")

    # Rank the excuses from the current run
    ranked_current_excuses = rank_excuses_data(current_run_excuses) # Pass the list to rank_excuses_data
    print("\n🏅 Ranked Excuses (Current Run):")
    for i, excuse_data in enumerate(ranked_current_excuses):
         print(f"{i+1}. Effectiveness: {excuse_data.effectiveness:.2f} - {excuse_data.excuse}")

    # Ask user to select an excuse for proofs
    selected_excuse_data = None
    while True:
        try:
            selection = int(input(f"\nSelect an excuse number (1-{len(current_run_excuses)}) from the GENERATED list above to generate proofs for, or enter 0 to skip proofs: "))
            if 0 <= selection <= len(current_run_excuses):
                break
            else:
                print("Invalid selection. Please enter a number within the range.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    new_excuses_to_save_dicts = [] # List to hold dictionaries of selected excuse for saving
    if selection > 0:
        selected_excuse_data = current_run_excuses[selection - 1]
        # print(f"Type of selected_excuse_data: {type(selected_excuse_data)}") # Added for debugging
        print(f"\nGenerating proofs for selected excuse: {selected_excuse_data.excuse}")

        proofs = {}
        # Chat log
        if input("Do you want a fake chat log for this excuse? (yes/no): ").strip().lower() == 'yes':
            chat_log_data = generate_fake_chat(selected_excuse_data.excuse)
            proofs['chat_log'] = chat_log_data # Store the data
            print("\n📱 Chat Log:")
            for msg in chat_log_data:
                print(f"[{msg['sender']} - {msg['time']}] {msg['message']}")

        # Medical note
        if input("Do you want a fake medical note? (yes/no): ").strip().lower() == 'yes':
            medical_note_text = generate_fake_medical_note()
            proofs['medical_note'] = medical_note_text # Store the text
            print("\n📄 Medical Note:")
            print(medical_note_text)

        # Location log
        if input("Do you want a fake location log? (yes/no): ").strip().lower() == 'yes':
            location_log_data = generate_fake_location_log()
            proofs['location_log'] = location_log_data # Store the data
            print("\n📍 Location Log (JSON):")
            print(json.dumps(location_log_data, indent=4))

        # Update the selected ExcuseData object with generated proofs
        selected_excuse_data.proofs = proofs
        new_excuses_to_save_dicts.append(selected_excuse_data.to_dict()) # Add selected excuse to the list for saving

    else:
        print("\nSkipping proof generation and saving for this run.")

    # Convert the selected excuse data to a DataFrame (will be empty if no excuse was selected)
    new_excuses_df = pd.DataFrame(new_excuses_to_save_dicts)

    # Append new data to the existing DataFrame
    all_excuses_df = pd.concat([all_excuses_df, new_excuses_df], ignore_index=True)

    # Save the combined DataFrame back to CSV
    save_excuses_to_csv(all_excuses_df)

    # Optional: Display the full history from the CSV
    print("\n=== Full Excuse History from CSV ===")
    display(load_excuses_from_csv())

Device set to use cpu


=== FLAN-T5 Excuse + Fake Proof Generator & CSV Saver ===
Loaded 1 existing excuses from excuse_history.csv
Enter a situation (e.g., I missed the test): I didn't come for an exam 
What tone should the apology have? (casual, formal, emotional - default emotional): formal

📝 Generated 5 Excuses:
1.  I just kept having to stay home.
2.  I got a full day off before the exam.
3.  I missed it
4.  I didn't want to go
5.  I was out of town.

🏅 Ranked Excuses (Current Run):
1. Effectiveness: 0.97 -  I just kept having to stay home.
2. Effectiveness: 0.91 -  I missed it
3. Effectiveness: 0.73 -  I got a full day off before the exam.
4. Effectiveness: 0.56 -  I was out of town.
5. Effectiveness: 0.53 -  I didn't want to go

Select an excuse number (1-5) from the GENERATED list above to generate proofs for, or enter 0 to skip proofs: 5

Generating proofs for selected excuse:  I was out of town.
Do you want a fake chat log for this excuse? (yes/no): no
Do you want a fake medical note? (yes/no): no


,Context,Tone,Excuse,Apology,Proofs,Calls,Effectiveness
0,missed project deadline,emotional,I'm trying to be smarter.,NaN,{},0,0.618033
1,I didn't come for an exam,formal,I was out of town.,NaN,{'location_log': {'location_log': [{'timestamp...,0,0.560939


In [ ]:
import random
import json
import pandas as pd
from datetime import datetime, timedelta, date # Import date explicitly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline # Import pipeline here
import torch

# === Constants ===

CSV_FILE = 'excuse_history.csv'
DISCLAIMER = "Disclaimer: This chat log is fictional and generated by an AI. It should not be used for deceptive purposes."

# === Excuse Data Structure ===
class ExcuseData:
    def __init__(self, context, tone, excuse, apology, proofs=None, calls=0, effectiveness=None):
        self.context = context
        self.tone = tone
        self.excuse = excuse
        self.apology = apology
        self.proofs = proofs if proofs is not None else {}
        self.calls = calls
        self.effectiveness = effectiveness

    def to_dict(self):
        return {
            "Context": self.context,
            "Tone": self.tone,
            "Excuse": self.excuse,
            "Apology": self.apology,
            "Proofs": json.dumps(self.proofs), # Convert proofs dictionary to JSON string
            "Calls": self.calls,
            "Effectiveness": self.effectiveness
        }

# === Load FLAN-T5 Model ===
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# === FLAN-T5 Excuse Generator ===
def generate_excuses(situation, num_excuses=5, max_length=50):
    prompt = f"Give {num_excuses} creative excuses for the following situation: {situation}"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_excuses,
            temperature=0.9,
            do_sample=True
        )

    excuses = []
    for output in outputs:
        decoded = tokenizer.decode(output, skip_special_tokens=True)
        # Prefix disclaimer
        excuses.append(f" {decoded}")
    return excuses

# Excuse History & Ranking (using the ExcuseData structure)
excuse_history_data = [] # Use this list to store ExcuseData objects for the current run

def add_excuse_data(excuse_data):
    excuse_history_data.append(excuse_data)

def rank_excuses_data(excuse_list): # Modified to accept a list
    # Rank based on the effectiveness stored in ExcuseData objects
    return sorted(excuse_list, key=lambda x: x.effectiveness if x.effectiveness is not None else -1, reverse=True)



# === Fictional Chat Generator ===
def generate_fake_chat(excuse):
    # Return the chat log data instead of printing
    return [
        {"sender": "Mom", "time": "8:42 AM", "message": "Where are you?? Your professor called 😡"},
        {"sender": "You", "time": "8:43 AM", "message": excuse},
        {"sender": "Mom", "time": "8:43 AM", "message": "Send a picture right now."},
        {"sender": "You", "time": "8:44 AM", "message": "Sent you one just now, look at the jam. I’ll be there ASAP."},
        {"sender": "System", "time": "Generated Now", "message": DISCLAIMER}
    ]


# === Medical Certificate Generator ===
def generate_fake_medical_note(patient_name="John Doe", reason=""):
    today = date.today() # Use date.today()
    issue_date = today.strftime("%B %d, %Y")
    rest_days = random.randint(2, 7)

    certificate_text = f"""
------------------------------------------------------------
                         MEDICAL CERTIFICATE
------------------------------------------------------------

To Whom It May Concern,

This is to certify that {patient_name} has been under my care and treatment
for the following medical reason: {reason}. The patient reported symptoms
requiring clinical attention and was advised to take adequate rest.

It is hereby recommended that {patient_name} be granted leave for a period
of {rest_days} days, starting from {issue_date}, in order to facilitate recovery.

Should you have any questions, you may contact my clinic directly.

Sincerely,


M.B.B.S., M.D.
Reg. No: {random.randint(100000, 999999)}
Clinic: Health First Multispeciality, Hyderabad
Date Issued: {issue_date}
------------------------------------------------------------
"""
    return certificate_text

# === Fake Location Log ===
def generate_fake_location_log():
    # Return the location log data instead of printing
    now = datetime.now()
    locations = [
        {
            "timestamp": now.isoformat(),
            "latitude": 12.9716,
            "longitude": 77.5946,

        },
        {
            "timestamp": (now + timedelta(minutes=90)).isoformat(),
            "latitude": 12.9353,
            "longitude": 77.6143,

        }
    ]
    return {

        "location_log": locations
    }

# === Guilt-Tripping Apology Generator (using pipeline from previous cells) ===
apology_generator = pipeline("text-generation", model="gpt2")

def generate_apology(context, tone="emotional"):
    prompt = f"I'm deeply sorry for {context}. I feel really {tone} about it because"
    # Ensure prompt is within a reasonable length for the model
    if len(prompt) > tokenizer.model_max_length:
        prompt = prompt[:tokenizer.model_max_length]

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    # Check if pad_token_id is None and set it to eos_token_id if it is
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Create attention mask
    attention_mask = inputs['input_ids'].ne(tokenizer.pad_token_id)

    with torch.no_grad():
        apology_outputs = model.generate(
            **inputs,
            max_length=50, # Limit apology length
            num_return_sequences=1,
            temperature=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            # attention_mask=attention_mask # Removed redundant attention mask
        )
    apology = tokenizer.decode(apology_outputs[0], skip_special_tokens=True)
    # Remove the prompt from the generated text
    apology = apology[len(prompt):].strip()
    return apology


# === CSV Handling Functions ===
def load_excuses_from_csv(filepath=CSV_FILE):
    try:
        df = pd.read_csv(filepath)
        # Convert the 'Proofs' column from JSON string back to dictionary
        if 'Proofs' in df.columns:
            df['Proofs'] = df['Proofs'].apply(lambda x: json.loads(x) if pd.notnull(x) else {})
        return df
    except FileNotFoundError:
        return pd.DataFrame(columns=["Context", "Tone", "Excuse", "Apology", "Proofs", "Calls", "Effectiveness"])

def save_excuses_to_csv(df, filepath=CSV_FILE):
    # Convert the 'Proofs' column (dictionaries) to JSON strings before saving
    if 'Proofs' in df.columns:
        df['Proofs'] = df['Proofs'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)
    df.to_csv(filepath, index=False)
    print(f"\nExcuse data saved to {filepath}")

# === Main Program ===
if __name__ == "__main__":
    print("=== FLAN-T5 Excuse + Fake Proof Generator & CSV Saver ===")

    # Load existing data from CSV
    all_excuses_df = load_excuses_from_csv()
    print(f"Loaded {len(all_excuses_df)} existing excuses from {CSV_FILE}")

    # Get user inputs for scenario and tone
    scenario = input("Enter a situation (e.g., I missed the test): ")
    # Add input for tone, as used in apology generation
    tone = input("What tone should the apology have? (casual, formal, emotional - default emotional): ") or "emotional"


    # Generate excuses with FLAN-T5
    generated_excuses_text = generate_excuses(scenario, num_excuses=5)

    print(f"\n📝 Generated {len(generated_excuses_text)} Excuses:")
    current_run_excuses = [] # List to hold ExcuseData objects for the current run

    for i, excuse_text in enumerate(generated_excuses_text):
        # Generate apology for each excuse (using the scenario as context)
        apology_text = generate_apology(scenario, tone)

        # Create an ExcuseData object for each generated excuse
        excuse_data = ExcuseData(
            context=scenario,
            tone=tone,
            excuse=excuse_text,
            apology=apology_text,
            effectiveness=random.uniform(0.5, 1.0) # Assign random effectiveness for now
        )
        current_run_excuses.append(excuse_data)
        print(f"{i+1}. {excuse_text}")

    # Rank the excuses from the current run
    ranked_current_excuses = rank_excuses_data(current_run_excuses) # Pass the list to rank_excuses_data
    print("\n🏅 Ranked Excuses (Current Run):")
    for i, excuse_data in enumerate(ranked_current_excuses):
         print(f"{i+1}. Effectiveness: {excuse_data.effectiveness:.2f} - {excuse_data.excuse}")

    # Ask user to select an excuse for proofs
    selected_excuse_data = None
    while True:
        try:
            selection = int(input(f"\nSelect an excuse number (1-{len(current_run_excuses)}) from the GENERATED list above to generate proofs for, or enter 0 to skip proofs: "))
            if 0 <= selection <= len(current_run_excuses):
                break
            else:
                print("Invalid selection. Please enter a number within the range.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    new_excuses_to_save_dicts = [] # List to hold dictionaries of selected excuse for saving
    if selection > 0:
        selected_excuse_data = current_run_excuses[selection - 1]
        # print(f"Type of selected_excuse_data: {type(selected_excuse_data)}") # Added for debugging
        print(f"\nGenerating proofs for selected excuse: {selected_excuse_data.excuse}")

        proofs = {}
        # Chat log
        if input("Do you want a fake chat log for this excuse? (yes/no): ").strip().lower() == 'yes':
            chat_log_data = generate_fake_chat(selected_excuse_data.excuse)
            proofs['chat_log'] = chat_log_data # Store the data
            print("\n📱 Chat Log:")
            for msg in chat_log_data:
                print(f"[{msg['sender']} - {msg['time']}] {msg['message']}")

        # Medical note
        if input("Do you want a fake medical note? (yes/no): ").strip().lower() == 'yes':
            medical_note_text = generate_fake_medical_note()
            proofs['medical_note'] = medical_note_text # Store the text
            print("\n📄 Medical Note:")
            print(medical_note_text)

        # Location log
        if input("Do you want a fake location log? (yes/no): ").strip().lower() == 'yes':
            location_log_data = generate_fake_location_log()
            proofs['location_log'] = location_log_data # Store the data
            print("\n📍 Location Log (JSON):")
            print(json.dumps(location_log_data, indent=4))

        # Update the selected ExcuseData object with generated proofs
        selected_excuse_data.proofs = proofs
        new_excuses_to_save_dicts.append(selected_excuse_data.to_dict()) # Add selected excuse to the list for saving

    else:
        print("\nSkipping proof generation and saving for this run.")

    # Convert the selected excuse data to a DataFrame (will be empty if no excuse was selected)
    new_excuses_df = pd.DataFrame(new_excuses_to_save_dicts)

    # Append new data to the existing DataFrame
    all_excuses_df = pd.concat([all_excuses_df, new_excuses_df], ignore_index=True)

    # Save the combined DataFrame back to CSV
    save_excuses_to_csv(all_excuses_df)

    # Optional: Display the full history from the CSV
    print("\n=== Full Excuse History from CSV ===")
    display(load_excuses_from_csv())

Device set to use cpu


=== FLAN-T5 Excuse + Fake Proof Generator & CSV Saver ===
Loaded 2 existing excuses from excuse_history.csv
Enter a situation (e.g., I missed the test): didn't mail in important files 
What tone should the apology have? (casual, formal, emotional - default emotional): casual

📝 Generated 5 Excuses:
1.  i rang in files
2.  I was ill and needed to send something
3.  lost a new CD.
4.  my boss says i forgot paperwork
5.  when I was in Italy

🏅 Ranked Excuses (Current Run):
1. Effectiveness: 0.99 -  i rang in files
2. Effectiveness: 0.93 -  my boss says i forgot paperwork
3. Effectiveness: 0.85 -  I was ill and needed to send something
4. Effectiveness: 0.80 -  when I was in Italy
5. Effectiveness: 0.72 -  lost a new CD.

Select an excuse number (1-5) from the GENERATED list above to generate proofs for, or enter 0 to skip proofs: 2

Generating proofs for selected excuse:  I was ill and needed to send something
Do you want a fake chat log for this excuse? (yes/no): no
Do you want a fake me

,Context,Tone,Excuse,Apology,Proofs,Calls,Effectiveness
0,missed project deadline,emotional,I'm trying to be smarter.,NaN,{},0,0.618033
1,I didn't come for an exam,formal,I was out of town.,NaN,{'location_log': {'location_log': [{'timestamp...,0,0.560939
2,didn't mail in important files,casual,I was ill and needed to send something,NaN,{'medical_note': ' ---------------------------...,0,0.849021
